In [0]:
%run "./Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("entityColumns", "")
dbutils.widgets.text("sourceSystem", "")

# Save the parameter to a variable:
entityName =       dbutils.widgets.get('entityName')
sourceSystemName = dbutils.widgets.get('sourceSystem')
columns =          json.loads(dbutils.widgets.get('entityColumns'))

In [0]:
entity_names = ['counterparty', 'currency', 'department', 'design', 'location', 'payment', 'payment_type', 'purchase_order', 'sales_order', 'staff', 'transaction']

In [0]:
# For each entity, create a dataFrame:
for entity in entity_names:
   globals()[f"df_{entity}"] = spark.read.format('parquet').load(BRONZE + f'/TotesysDB/{entity}/{datetime.datetime.now().strftime("%Y/%m/%d")}')

In [0]:
# Iterate through entities to create temp views:
for entity in entity_names:
    # Create or replace tempView:
    globals()[f'df_{entity}'].createOrReplaceTempView(f"tv_{entity}")

In [0]:
if entityName == 'design':
    
    # Define Dynamic Query:
    query = f"""
    SELECT {columns[0]['dataType']}(design_id) AS {columns[0]['columnName']}
    , {columns[1]['dataType']}(created_at) AS {columns[1]['columnName']}
    , {columns[2]['dataType']}(last_updated) AS {columns[2]['columnName']}
    , {columns[3]['dataType']}(design_name) AS {columns[3]['columnName']}
    FROM tv_design
    """
    # Execute Query:
    df = spark.sql(query)
    
    
    

In [0]:
# Check data exists:
if df.count() > 0:
    folder_path = datetime.datetime.now().strftime("%Y/%m/%d")

    # Set location for parquet:
    location = SILVER + f"/{sourceSystemName}/{entityName}/{folder_path}/"
    
    # Write parquet to data lake:
    df.write.mode('overwrite').format('delta').option('header', 'true').save(location)
else:
    dbutils.notebook.exit(-1)
